## Import

In [1]:
! pip install rouge

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
import numpy as np

import re
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
import difflib

import rouge

from IPython.core.display import display, HTML

In [3]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
testd = pd.read_csv("/content/drive/MyDrive/miniproject/testd.csv",index_col="Unnamed: 0")#.drop(columns =["text_clean","y_clean"])
teste = pd.read_csv("/content/drive/MyDrive/miniproject/teste.csv",index_col="Unnamed: 0")#.drop(columns =["text_clean","y_clean"])
textscores = pd.read_csv("/content/drive/MyDrive/miniproject/textscores.csv",index_col="Unnamed: 0")
bleuscore = pd.read_csv("/content/drive/MyDrive/miniproject/bleuscore.csv").drop(columns =["Unnamed: 0"])
t5ea = pd.read_csv("/content/drive/MyDrive/miniproject/t5ea.csv").drop(columns =["Unnamed: 0"])

In [5]:
pd.set_option('display.max_colwidth', None)

## Visualization 

In [6]:
def utils_split_sentences(a, b):
    ## find clean matches
    match = difflib.SequenceMatcher(isjunk=None, a=a, b=b, autojunk=True)
    lst_match = [block for block in match.get_matching_blocks() if block.size > 20]
    
    ## difflib didn't find any match
    if len(lst_match) == 0:
        lst_a, lst_b = nltk.sent_tokenize(a), nltk.sent_tokenize(b)
    
    ## work with matches
    else:
        first_m, last_m = lst_match[0], lst_match[-1]

        ### a
        string = a[0 : first_m.a]
        lst_a = [t for t in nltk.sent_tokenize(string)]
        for n in range(len(lst_match)):
            m = lst_match[n]
            string = a[m.a : m.a+m.size]
            lst_a.append(string)
            if n+1 < len(lst_match):
                next_m = lst_match[n+1]
                string = a[m.a+m.size : next_m.a]
                lst_a = lst_a + [t for t in nltk.sent_tokenize(string)]
            else:
                break
        string = a[last_m.a+last_m.size :]
        lst_a = lst_a + [t for t in nltk.sent_tokenize(string)]

        ### b
        string = b[0 : first_m.b]
        lst_b = [t for t in nltk.sent_tokenize(string)]
        for n in range(len(lst_match)):
            m = lst_match[n]
            string = b[m.b : m.b+m.size]
            lst_b.append(string)
            if n+1 < len(lst_match):
                next_m = lst_match[n+1]
                string = b[m.b+m.size : next_m.b]
                lst_b = lst_b + [t for t in nltk.sent_tokenize(string)]
            else:
                break
        string = b[last_m.b+last_m.size :]
        lst_b = lst_b + [t for t in nltk.sent_tokenize(string)]
    
    return lst_a, lst_b

In [7]:
def display_string_matching(a, b, both=True, sentences=True, titles=[]):
    if sentences is True:
        lst_a, lst_b = utils_split_sentences(a, b)
    else:
        lst_a, lst_b = a.split(), b.split()       
    
    ## highlight a
    first_text = []
    for i in lst_a:
        if re.sub(r'[^\w\s]', '', i.lower()) in [re.sub(r'[^\w\s]', '', z.lower()) for z in lst_b]:
            first_text.append('<span style="background-color:rgba(255,215,0,0.3);">' + i + '</span>')
        else:
            first_text.append(i)
    first_text = ' '.join(first_text)
    
    ## highlight b
    second_text = []
    if both is True:
        for i in lst_b:
            if re.sub(r'[^\w\s]', '', i.lower()) in [re.sub(r'[^\w\s]', '', z.lower()) for z in lst_a]:
                second_text.append('<span style="background-color:rgba(255,215,0,0.3);">' + i + '</span>')
            else:
                second_text.append(i)
    else:
        second_text.append(b) 
    second_text = ' '.join(second_text)
    
    ## concatenate
    if len(titles) > 0:
        first_text = "<strong>"+titles[0]+"</strong><br>"+first_text
    if len(titles) > 1:
        second_text = "<strong>"+titles[1]+"</strong><br>"+second_text
    else:
        second_text = "---"*65+"<br><br>"+second_text
    final_text = first_text +'<br><br>'+ second_text
    return final_text

## Comparision

In [8]:
testd.style.set_properties(**{'vertical-align': 'top', 'text-align': 'left'}) #cnn news

In [9]:
teste.style.set_properties(**{'vertical-align': 'top', 'text-align': 'left'}) #essay,story,

,text,y,text_clean,y_clean,ey,nlpred,t5pred,trpred
0,"Once upon a time, there was a young girl named Alice who lived in a small village near the forest. One day, while playing in the woods, she stumbled upon a rabbit hole. Curious, she followed the rabbit down the hole and found herself in a strange and magical world. She met many unusual characters, such as the Cheshire Cat and the Mad Hatter, and went on incredible adventures. Eventually, Alice found her way back home, but she never forgot the wonderful world she had discovered.",Alice is a young girl who falls down a rabbit hole and enters a strange and magical world. She meets many unusual characters and has incredible adventures before eventually finding her way back home.,upon time young girl named alice lived small village near forest one day playing wood stumbled upon rabbit hole curious followed rabbit hole found strange magical world met many unusual character cheshire cat mad hatter went incredible adventure eventually alice found way back home never forgot wonderful world discovered,alice young girl fall rabbit hole enters strange magical world meet many unusual character incredible adventure eventually finding way back home,"Once upon a time, there was a young girl named Alice who lived in a small village near the forest.Curious, she followed the rabbit down the hole and found herself in a strange and magical world.","Once upon a time, there was a young girl named Alice who lived in a small village near the forest.Curious, she followed the rabbit down the hole and found herself in a strange and magical world.Eventually, Alice found her way back home, but she never forgot the wonderful world she had discovered.","one day, while playing in the woods, Alice stumbled upon a rabbit hole. curiously, she followed the rabbit down the hole and found herself in an amazing world. Eventually she found her way back home, but never forgot the wonderful world she had discovered.","Curious, she followed the rabbit down the hole and found herself in a strange and magical world. One day, while playing in the woods, she stumbled upon a rabbit hole."
1,"Once upon a time, in a small village nestled in the hills, there was a young girl named Lily. She lived with her parents and spent most of her time playing in the fields and forests surrounding their home. One day, while exploring a nearby creek, Lily stumbled upon a small, injured bird. She took the bird back to her home and cared for it until it was able to fly again. From that day on, Lily became known as the village's protector of injured animals, and she spent much of her time nursing them back to health.","Lily is a young girl who lives in a small village in the hills. She becomes known as the protector of injured animals after caring for a small, injured bird she found while exploring a nearby creek.",upon time small village nestled hill young girl named lily lived parent spent time playing field forest surrounding home one day exploring nearby creek lily stumbled upon small injured bird took bird back home cared able fly day lily became known village protector injured animal spent much time nursing back health,lily young girl life small village hill becomes known protector injured animal caring small injured bird found exploring nearby creek,"Once upon a time, in a small village nestled in the hills, there was a young girl named Lily.From that day on, Lily became known as the village's protector of injured animals, and she spent much of her time nursing them back to health.","From that day on, Lily became known as the village's protector of injured animals, and she spent much of her time nursing them back to health.Once upon a time, in a small village nestled in the hills, there was a young girl named Lily.One day, while exploring a nearby creek, Lily stumbled upon a small, injured bird.",a small village girl named Lily cared for injured animals in the hills. she took the bird back to her home and nursed it until it was able t

## ROUGE 
Recall-Oriented Understudy for Gisting Evaluation


In [10]:
import rouge
def evaluate_summary(y_test, predicted):    
   rouge_score = rouge.Rouge()    
   scores = rouge_score.get_scores(y_test, predicted, avg=True)       
   score_1 = round(scores['rouge-1']['f'], 2)    
   score_2 = round(scores['rouge-2']['f'], 2)    
   score_L = round(scores['rouge-l']['f'], 2)    
   print("rouge1:", score_1, "| rouge2:", score_2, "| rougeL:", score_L, "--> avg rouge:", round(np.mean([score_1,score_2,score_L]), 2))
   return np.mean([score_1,score_2,score_L])

In [11]:
for i in testd.index:
    evaluate_summary(testd['ey'][i], testd['t5pred'][i])
print("teste")
for i in teste.index:
    evaluate_summary(teste['ey'][i], teste['t5pred'][i])

rouge1: 0.35 | rouge2: 0.23 | rougeL: 0.35 --> avg rouge: 0.31
rouge1: 0.19 | rouge2: 0.07 | rougeL: 0.17 --> avg rouge: 0.14
rouge1: 0.19 | rouge2: 0.06 | rougeL: 0.19 --> avg rouge: 0.15
rouge1: 0.23 | rouge2: 0.07 | rougeL: 0.23 --> avg rouge: 0.18
rouge1: 0.18 | rouge2: 0.06 | rougeL: 0.18 --> avg rouge: 0.14
rouge1: 0.26 | rouge2: 0.11 | rougeL: 0.26 --> avg rouge: 0.21
rouge1: 0.27 | rouge2: 0.12 | rougeL: 0.26 --> avg rouge: 0.22
rouge1: 0.19 | rouge2: 0.09 | rougeL: 0.19 --> avg rouge: 0.16
rouge1: 0.47 | rouge2: 0.2 | rougeL: 0.39 --> avg rouge: 0.35
rouge1: 0.14 | rouge2: 0.03 | rougeL: 0.14 --> avg rouge: 0.1
rouge1: 0.26 | rouge2: 0.16 | rougeL: 0.26 --> avg rouge: 0.23
rouge1: 0.24 | rouge2: 0.09 | rougeL: 0.23 --> avg rouge: 0.19
rouge1: 0.11 | rouge2: 0.01 | rougeL: 0.09 --> avg rouge: 0.07
rouge1: 0.23 | rouge2: 0.05 | rougeL: 0.21 --> avg rouge: 0.16
rouge1: 0.43 | rouge2: 0.21 | rougeL: 0.42 --> avg rouge: 0.35
rouge1: 0.18 | rouge2: 0.01 | rougeL: 0.15 --> avg rouge:

In [12]:
#ROUGE Scores
#Tested against sumy TextRank for extractive
textscores

,id,set,nLargest(Avg),TextRank(Avg),T5(Avg),nLargest(r1),nLargest(r2),nLargest(rL),TextRank(r1),TextRank(r2),TextRank(rL),T5(r1),T5(r2),T5(rL)
0,0,d,0.656667,0.853333,0.086667,0.675159,0.618605,0.668790,0.868035,0.817797,0.868035,0.129032,0.000000,0.129032
1,1,d,0.746667,0.843333,0.313333,0.766990,0.700775,0.766990,0.862559,0.811377,0.862559,0.367816,0.198020,0.367816
2,2,d,0.650000,0.816667,0.263333,0.674699,0.613757,0.668675,0.838710,0.780255,0.833333,0.354430,0.139535,0.303797
3,3,d,0.520000,0.623333,0.210000,0.559671,0.439628,0.559671,0.658537,0.548673,0.658537,0.257143,0.112676,0.257143
4,4,d,0.773333,0.706667,0.153333,0.793028,0.735178,0.788671,0.730159,0.661290,0.725624,0.206897,0.040404,0.206897
5,5,d,0.633333,0.780000,0.193333,0.673797,0.558059,0.673797,0.803150,0.737013,0.797900,0.229167,0.123894,0.229167
6,10,d,0.526667,0.733333,0.316667,0.574307,0.450331,0.559194,0.761468,0.693997,0.752294,0.386364,0.203704,0.363636
7,100,d,0.546667,0.656667,0.310000,0.583851,0.484848,0.577640,0.691358,0.604000,0.679012,0.411765,0.135135,0.382353
8,293,d,0.740000,1.000000,0.126667,0.763636,0.696970,0.763636,1.000000,1.000000,1.000000,0.190476,0.000000,0.190476
9,951,d,0.580000,0.853333,0.173333,0.605317,0.531507,0.601227,0.870544,0.824845,0.870544,0.235294,0.093023,0.188235


In [13]:
t5ea

,id,set,T5(Avg),T5(r1),T5(r2),T5(rL),T5e(Avg),T5e(r1),T5e(r2),T5e(rL)
0,0,d,0.086667,0.129032,0.000000,0.129032,0.310000,0.352423,0.231023,0.352423
1,1,d,0.313333,0.367816,0.198020,0.367816,0.143333,0.187266,0.074627,0.172285
2,2,d,0.263333,0.354430,0.139535,0.303797,0.146667,0.193833,0.061111,0.193833
3,3,d,0.210000,0.257143,0.112676,0.257143,0.176667,0.229885,0.069565,0.229885
4,4,d,0.153333,0.206897,0.040404,0.206897,0.140000,0.177936,0.058036,0.177936
5,5,d,0.193333,0.229167,0.123894,0.229167,0.210000,0.256000,0.109091,0.256000
6,10,d,0.316667,0.386364,0.203704,0.363636,0.216667,0.272059,0.121951,0.264706
7,100,d,0.310000,0.411765,0.135135,0.382353,0.156667,0.194872,0.088737,0.194872
8,293,d,0.126667,0.190476,0.000000,0.190476,0.353333,0.470588,0.203390,0.392157
9,951,d,0.173333,0.235294,0.093023,0.188235,0.103333,0.135048,0.026549,0.135048


## BLEU
Bilingual Evaluation Understudy, is a metric for comparing a candidate translation to one or more reference translations.

In [15]:
from nltk.translate.bleu_score import sentence_bleu,corpus_bleu
#293 9838 1000
i = 9838
reference = [testd['ey'][i],testd['y'][i],testd['y_clean'][i]]
candidate = testd['trpred'][i]
print('Individual 1-gram: %f' % sentence_bleu(reference, candidate, weights=(1, 0, 0, 0)))
print('Individual 2-gram: %f' % sentence_bleu(reference, candidate, weights=(0, 1, 0, 0)))
print('Individual 3-gram: %f' % sentence_bleu(reference, candidate, weights=(0, 0, 1, 0)))
print('Individual 4-gram: %f' % sentence_bleu(reference, candidate, weights=(0, 0, 0, 1)))
print('Unigram,Bigram score: %f' % sentence_bleu(reference, candidate, weights=(1, 1, 0, 0)))
print('Unigram,Bigram,Trigram score: %f' % sentence_bleu(reference, candidate, weights=(1, 1, 1, 0)))
print('Unigram,Bigram,Trigram, 4-gram score: %f' % sentence_bleu(reference, candidate, weights=(1, 1, 1, 1)))
print('BLEU-2 score: %f' % sentence_bleu(reference, candidate, weights=(0.5, 0.5, 0, 0)))
print('BLEU-3 score: %f' % sentence_bleu(reference, candidate, weights=(0.3333, 0.3333, 0.3333, 0)))
print('BLEU-4 score: %f' % sentence_bleu(reference, candidate, weights=(0.25, 0.25, 0.25, 0.25)))

print('Individual 1-gram: %f' % corpus_bleu([reference], [candidate], weights=(1, 0, 0, 0)))
print('Individual 2-gram: %f' % corpus_bleu([reference], [candidate], weights=(0, 1, 0, 0)))
print('Individual 3-gram: %f' % corpus_bleu([reference], [candidate], weights=(0, 0, 1, 0)))
print('Individual 4-gram: %f' % corpus_bleu([reference], [candidate], weights=(0, 0, 0, 1)))
print('Unigram,Bigram score: %f' % corpus_bleu([reference], [candidate], weights=(1, 1, 0, 0)))
print('Unigram,Bigram,Trigram score: %f' % corpus_bleu([reference], [candidate], weights=(1, 1, 1, 0)))
print('Unigram,Bigram,Trigram, 4-gram score: %f' % corpus_bleu([reference], [candidate], weights=(1, 1, 1, 1)))
print('BLEU-2 score: %f' % corpus_bleu([reference], [candidate], weights=(0.5, 0.5, 0, 0)))
print('BLEU-3 score: %f' % corpus_bleu([reference], [candidate], weights=(0.3333, 0.3333, 0.3333, 0)))
print('BLEU-4 score: %f' % corpus_bleu([reference], [candidate], weights=(0.25, 0.25, 0.25, 0.25)))

Individual 1-gram: 0.855405
Individual 2-gram: 0.745602
Individual 3-gram: 0.594851
Individual 4-gram: 0.512890
Unigram,Bigram score: 0.637792
Unigram,Bigram,Trigram score: 0.379391
Unigram,Bigram,Trigram, 4-gram score: 0.194586
BLEU-2 score: 0.798619
BLEU-3 score: 0.723952
BLEU-4 score: 0.664168
Individual 1-gram: 0.855405
Individual 2-gram: 0.745602
Individual 3-gram: 0.594851
Individual 4-gram: 0.512890
Unigram,Bigram score: 0.637792
Unigram,Bigram,Trigram score: 0.379391
Unigram,Bigram,Trigram, 4-gram score: 0.194586
BLEU-2 score: 0.798619
BLEU-3 score: 0.723952
BLEU-4 score: 0.664168


In [16]:
#tr
bleuscore[bleuscore["Model"]=="tr"]

,id,set,Model,BLEU-4,BLEU-1,BLEU-2,BLEU-3,i_2-gram,i_3-gram,i_4-gram,2-gram,3-gram,4-gram
28,0,d,tr,0.855381,0.926479,0.898594,0.873899,0.871548,0.826526,0.802150,0.854459,0.747329,0.634354
29,1,d,tr,0.829964,0.892623,0.869378,0.847196,0.846738,0.804516,0.780343,0.842367,0.755300,0.656885
30,2,d,tr,0.867092,0.961573,0.925205,0.891585,0.890212,0.827966,0.797577,0.866363,0.726000,0.586048
31,3,d,tr,0.665123,0.815408,0.765595,0.710128,0.718824,0.610959,0.546509,0.716739,0.535470,0.357846
32,4,d,tr,0.775288,0.887689,0.854083,0.812079,0.821749,0.734168,0.674616,0.811705,0.663119,0.497790
33,5,d,tr,0.871646,0.963543,0.931445,0.898524,0.900416,0.836131,0.795741,0.873788,0.735822,0.589707
34,10,d,tr,0.830121,0.949490,0.906855,0.863914,0.866135,0.784035,0.736469,0.822387,0.644780,0.474860
35,100,d,tr,0.743764,0.885704,0.834325,0.782075,0.785926,0.687186,0.639728,0.778749,0.598685,0.428471
36,293,d,tr,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
37,951,d,tr,0.870591,0.931659,0.908799,0.887017,0.886500,0.845007,0.823115,0.882644,0.797069,0.701143


In [17]:
#nlargest
bleuscore[bleuscore["Model"]=="nl"]

,id,set,Model,BLEU-4,BLEU-1,BLEU-2,BLEU-3,i_2-gram,i_3-gram,i_4-gram,2-gram,3-gram,4-gram
0,0,d,nl,0.618481,0.716243,0.687235,0.649900,0.659401,0.581205,0.533046,0.650700,0.521052,0.382663
1,1,d,nl,0.764984,0.841988,0.816956,0.788008,0.792669,0.733153,0.699870,0.780711,0.669542,0.548135
2,2,d,nl,0.666321,0.747411,0.723741,0.692404,0.700821,0.633741,0.593822,0.690671,0.577150,0.451908
3,3,d,nl,0.519269,0.678255,0.630574,0.567866,0.586245,0.460537,0.397037,0.582333,0.392766,0.228383
4,4,d,nl,0.841413,0.923490,0.895703,0.865984,0.868752,0.809471,0.771805,0.852915,0.733981,0.602241
5,5,d,nl,0.728295,0.827145,0.797310,0.759994,0.768550,0.690521,0.640913,0.750043,0.611076,0.462090
6,10,d,nl,0.596740,0.736772,0.697282,0.643319,0.659909,0.547598,0.476281,0.650554,0.476663,0.303767
7,100,d,nl,0.709281,0.874815,0.822780,0.759907,0.773840,0.648207,0.576755,0.755704,0.546826,0.352066
8,293,d,nl,0.990338,1.000000,0.996810,0.993590,0.993631,0.987179,0.980645,0.993631,0.980892,0.961907
9,951,d,nl,0.607457,0.705191,0.682120,0.643033,0.659803,0.571450,0.512111,0.648399,0.516348,0.368491


In [18]:
#T5
bleuscore[bleuscore["Model"]=="t5"]

,id,set,Model,BLEU-4,BLEU-1,BLEU-2,BLEU-3,i_2-gram,i_3-gram,i_4-gram,2-gram,3-gram,4-gram
56,0,d,t5,0.869507,0.993958,0.964731,0.919708,0.936364,0.835866,0.734756,0.930706,0.777946,0.571600
57,1,d,t5,0.838429,0.993548,0.967301,0.907478,0.941748,0.798701,0.661238,0.935672,0.747322,0.494158
58,2,d,t5,0.876776,1.000000,0.974778,0.925620,0.950192,0.834615,0.745174,0.950192,0.793045,0.590956
59,3,d,t5,0.754142,0.993939,0.950424,0.848734,0.908815,0.676829,0.529052,0.903307,0.611384,0.323454
60,4,d,t5,0.826314,0.997085,0.974904,0.906206,0.953216,0.782991,0.626471,0.950437,0.744184,0.466209
61,5,d,t5,0.828080,0.992042,0.969162,0.904054,0.946809,0.786667,0.636364,0.939274,0.738896,0.470206
62,10,d,t5,0.904867,1.000000,0.984874,0.950952,0.969977,0.886574,0.779582,0.969977,0.859956,0.670407
63,100,d,t5,0.739036,0.838890,0.825498,0.783361,0.812319,0.705430,0.620549,0.812319,0.683086,0.505297
64,293,d,t5,0.728637,0.973214,0.883360,0.795635,0.801802,0.645455,0.559633,0.780325,0.503664,0.281867
65,951,d,t5,0.782495,0.925347,0.915278,0.855852,0.905318,0.748324,0.598041,0.905318,0.732127,0.473165


## nLargest

In [19]:
for i in testd.index:
    match = display_string_matching(testd['ey'][i], testd['nlpred'][i], both=True, sentences=False, titles=["Real Summary", "Predicted Summary"])
    display(HTML(match))
    print()

In [20]:
for i in teste.index:
    match = display_string_matching(teste['ey'][i], teste['nlpred'][i], both=True, sentences=False, titles=["Real Summary", "Predicted Summary"])
    display(HTML(match))
    print()

## TextRank

In [21]:
for i in testd.index:
    match = display_string_matching(testd['ey'][i], testd['trpred'][i], both=True, sentences=False, titles=["Real Summary", "Predicted Summary"])
    display(HTML(match))
    print()

In [22]:
for i in teste.index:
    match = display_string_matching(teste['ey'][i], teste['trpred'][i], both=True, sentences=False, titles=["Real Summary", "Predicted Summary"])
    display(HTML(match))
    print()

## T5

In [23]:
for i in testd.index:
    match = display_string_matching(testd['y'][i], testd['t5pred'][i], both=True, sentences=False, titles=["Real Summary", "Predicted Summary"])
    display(HTML(match))
    print()

In [24]:
for i in teste.index:
    match = display_string_matching(teste['y'][i], teste['t5pred'][i], both=True, sentences=False, titles=["Real Summary", "Predicted Summary"])
    display(HTML(match))
    print()